# From Pandas to PostgreSQL: Bulk Insert with execute_values()

*By Naysan Saran, May 2020.*

## 1 - Introduction

In this tutorial we will go through all the steps required to 

- turn a csv into a pandas dataframe
- create the corresponding PostgreSQL database and table
- insert the pandas dataframe in the PostgreSQL table using execute_values()

The data for this tutorial is freely available on https://datahub.io/core/global-temp, but you will also find it in the data/ directory of this github repository. 

## 2 - From csv file to pandas dataframe

In [1]:
import pandas as pd

csv_file = "../data/global-temp-monthly.csv"
df = pd.read_csv(csv_file)
print("Total number of rows = %s" % len(df.index))
df.head(3)

Total number of rows = 3288


,Source,Date,Mean
0,GCAG,2016-12-06,0.7895
1,GISTEMP,2016-12-06,0.8100
2,GCAG,2016-11-06,0.7504


In [2]:
df = df.rename(columns={
    "Source": "source", 
    "Date": "datetime",
    "Mean": "mean_temp"
})
df.head(3)

,source,datetime,mean_temp
0,GCAG,2016-12-06,0.7895
1,GISTEMP,2016-12-06,0.8100
2,GCAG,2016-11-06,0.7504


## 3 - PostgreSQL database, table and user setup

First we create the database. I'm assuming you already have PostgreSQL installed on your system. Otherwise you can refer to this link first https://www.postgresql.org/download/.

Creating the database - Ubuntu command line instructions 

For the sake of simplicity, we are going to create one table only to store everything.

Lastly, let's create a user and give them access to our new table.

Last permission to grant, so 'myuser' can autoincrement the 'id' primary key without having to specify it

## 4 - Basic Python functions to access the database

Alright back to Python. Here are all the functions we will need. For a complete, functioning code, please refer to the src/ subdirectory.

In [3]:
import psycopg2
import psycopg2.extras as extras
import numpy as np

Fist, let's specify the connection parameters as a Python dictionary. The database, username and password will be the same that we created in part 3.

In [4]:
param_dic = {
    "host"      : "localhost",
    "database"  : "globaldata",
    "user"      : "myuser",
    "password"  : "Passw0rd"
}

This function will allow us to connect to the database

In [5]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn


Testing the database connection

In [6]:
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


Function to execute any query in the database. Will come in handy later as we test multiple insert tactics.

In [7]:
def execute_query(conn, query):
    """ Execute a single query """
    
    ret = 0 # Return value
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1

    # If this was a select query, return the result
    if 'select' in query.lower():
        ret = cursor.fetchall()
    cursor.close()
    return ret

## Bulk Insert with execute_values()

In [8]:
def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()


# Run the execute_many strategy
execute_values(conn, df, 'MonthlyTemp')

execute_values() done


In [9]:
# Check that the values were indeed inserted
execute_query(conn, "select count(*) from MonthlyTemp;")

[(3288,)]

In [10]:
# (Optional) Clear the table
execute_query(conn, "delete from MonthlyTemp where true;")

0

In [12]:
# Close the connection
conn.close()